In [ ]:
import ollama
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [ ]:
class Website:

    def __init__(self, url):

        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "Website without title"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
def prompt_for(website):
    prompt = f"Here is a website for you to summarize. Its title is {website.title}."
    prompt += "\nPlease provide a short summary of the website. If it contains any announcements, summarize those, too."
    prompt += f"\nHere is the Website:\n\n{website.text}"

    return prompt

In [ ]:
MODEL = "llama3.2"
website = Website("https://edwarddonner.com")

def messages_for(website):
    messages = [
        {"role": "system", "content": "You are a powerful, friendly, and helpful website summarization assistant. \
    You are given a website and summarize its content succinctly. You format your answer in markdown."},
        {"role": "user", "content": prompt_for(website)}
    ]

    return messages

In [ ]:
def display_md_summary(website):
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    display(Markdown(response['message']['content']))

In [ ]:
display_md_summary(website)